# Transformer麻雀AI 開発ワークフロー (緊急調査モード)

**【目的】**
データ生成過程におけるサンプル棄却率が異常に高い問題の原因を特定する。
パーサーの手牌シミュレーションにズレが生じていないかを徹底的に検証する。

## 1. 環境設定

In [7]:
!pip install mahjong
from google.colab import drive
drive.mount('/content/drive')

# ご指定のプロジェクトパスに移動します
%cd "/content/drive/MyDrive/いろいろ/麻雀AI/googlecolab/senasJanAI"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/いろいろ/麻雀AI/googlecolab/senasJanAI


## 2. データ準備
調査対象の牌譜ログをローカル環境にコピーします。

In [8]:
import os
import zipfile

# ご自身の環境に合わせて、牌譜ログが入ったzipファイル名を指定してください
ZIP_FILE_PATH = 'data.zip'
EXTRACT_DIR = '/content/data'

print(f"Copying '{ZIP_FILE_PATH}' to local environment...")
!cp -f "{ZIP_FILE_PATH}" /content/

print(f"Unzipping files to '{EXTRACT_DIR}'...")
if os.path.isdir(EXTRACT_DIR):
    !rm -rf {EXTRACT_DIR}
os.makedirs(EXTRACT_DIR, exist_ok=True)
with zipfile.ZipFile(f'/content/{ZIP_FILE_PATH}', 'r') as zip_ref:
    zip_ref.extractall(EXTRACT_DIR)

# zipファイル内に'data'ディレクトリがネストされている場合に対応
NESTED_DATA_PATH = os.path.join(EXTRACT_DIR, 'data')
if os.path.isdir(NESTED_DATA_PATH):
    print(f"Correcting nested data directory...")
    !rsync -a {NESTED_DATA_PATH}/ {EXTRACT_DIR}/
    !rm -rf {NESTED_DATA_PATH}

print("\nChecking for unzipped files (showing first 5):")
!ls -l {EXTRACT_DIR} | head -n 6

Copying 'data.zip' to local environment...
Unzipping files to '/content/data'...
Correcting nested data directory...

Checking for unzipped files (showing first 5):
total 311448
-rw-r--r-- 1 root root  2303 Sep 29 17:17 20061028gm-0001-0000-134c5173&tw=0.mjlog
-rw-r--r-- 1 root root  1795 Sep 29 17:17 20061028gm-0001-0000-1af63a72&tw=0.mjlog
-rw-r--r-- 1 root root  2199 Sep 29 17:17 20061028gm-0001-0000-32237e64&tw=0.mjlog
-rw-r--r-- 1 root root  3087 Sep 29 17:17 20061030gm-0001-0000-366b66c4&tw=3.mjlog
-rw-r--r-- 1 root root  2578 Sep 29 17:17 20061030gm-0001-0000-63cb6bfc&tw=2.mjlog


## 3. (重要) 状態シミュレータによる徹底調査

1つの牌譜ファイルを指定し、イベントごとにシミュレータが認識している全プレイヤーの手牌を詳細に出力します。
このログと実際の牌譜を見比べることで、どのタイミングで手牌の同期がズレるのかを正確に特定できます。

In [11]:
# 調査したい牌譜ログのファイル名をここに指定してください
# 指定しない場合、dataフォルダからランダムに1つ選ばれます
TARGET_LOG_FILE = ""

import os

if TARGET_LOG_FILE and os.path.exists(f'/content/data/{TARGET_LOG_FILE}'):
    test_file_path = f'/content/data/{TARGET_LOG_FILE}'
    print(f"Investigating specified file: {TARGET_LOG_FILE}")
else:
    print("No specific file targeted, finding a random log file...")
    test_file = !find /content/data -name "*.mjlog" -print -quit
    if not test_file:
        test_file = !find /content/data -name "*.gz" -print -quit

    if test_file:
        test_file_path = test_file[0]
    else:
        test_file_path = None

if test_file_path:
    print(f"Target for investigation: {os.path.basename(test_file_path)}")
    # 調査スクリプトを実行します
    !python -m src.transformer.investigate_state "{test_file_path}"
else:
    print("No .mjlog or .gz files found for investigation in /content/data")

ストリーミング出力は最後の 5000 行に切り捨てられました。
{'type': 'start_game'}

--- Simulator State (After Event) ---
  P0 Hand (0枚): 
  P1 Hand (0枚): 
  P2 Hand (0枚): 
  P3 Hand (0枚): 
------------------------------------------

==================== Event 2 ====================
{'dora_marker': 50,
 'haipai': [[8, 20, 37, 44, 53, 57, 61, 86, 89, 104, 113, 115, 119],
            [3, 4, 7, 12, 13, 31, 56, 80, 81, 97, 107, 117, 127],
            [2, 6, 9, 18, 25, 35, 41, 43, 55, 63, 82, 114, 131],
            [16, 17, 23, 27, 32, 47, 69, 88, 99, 100, 120, 122, 135]],
 'oya': 0,
 'type': 'start_kyoku'}

--- Simulator State (After Event) ---
  P0 Hand (13枚): 36m13567p459sSSW
  P1 Hand (13枚): 122448m6p3379sWH
  P2 Hand (13枚): 123579m2257p3sSG
  P3 Hand (13枚): 55679m39p578sNNC
------------------------------------------

==================== Event 3 ====================
{'actor': 0, 'pai': 5, 'type': 'tsumo'}

--- Simulator State (After Event) ---
  P0 Hand (14枚): 236m13567p459sSSW
  P1 Hand (13枚): 122448m6p3379sWH
 

## 4. 全データセットの生成
（原因が特定・修正されるまで、このステップは実行しないでください）

In [ ]:
!python -m src.transformer.generate_data

## 5. AIモデルの学習

In [ ]:
!python -m src.transformer.train_transformer

## 6. 学習済みAIによる予測

In [ ]:
!python -m src.transformer.predict_transformer